In [2]:
import yaml
from py2neo import Graph
from dotenv import load_dotenv
import os

# Load environment variables
load_dotenv()
neo4j_password = os.getenv('NEO4J_PASSWORD')

# Load YAML data from file
with open('agents.yaml', 'r') as file:
    data = yaml.safe_load(file)

# Connect to the Neo4j database
graph = Graph("neo4j://localhost:7687", auth=("neo4j", neo4j_password))

# Clear existing data (optional, be cautious with this)
graph.run("MATCH (n) DETACH DELETE n")

# Create nodes and relationships based on the YAML file
for node, properties in data.items():
    name = properties['Name']
    desc = properties['Desc']
    parents = properties['parent']

    # Create the node
    query = f"CREATE (n:{name} {{Name: '{name}', Desc: '{desc}'}})"
    graph.run(query)

    # Handle parent relationships
    if parents and parents != []:
        for parent in parents:
            # Create relationships with parent
            parent_query = f"MATCH (p:{parent}), (c:{name}) CREATE (p)-[:PARENT_OF]->(c)"
            graph.run(parent_query)
            child_query = f"MATCH (p:{parent}), (c:{name}) CREATE (c)-[:CHILD_OF]->(p)"
            graph.run(child_query)

print("Data imported successfully.")

Data imported successfully.


In [26]:
def get_connected_nodes(node_name):
    # Define the Cypher query to find connected nodes and return both Name and Desc properties
    query = """
    MATCH (n {Name: $node_name})-[:CHILD_OF|PARENT_OF]-(connected_nodes)
    RETURN connected_nodes.Name AS name, connected_nodes.Desc AS desc
    """
    
    # Execute the query
    results = graph.run(query, node_name=node_name)
    
    # Extract the Name and Desc of each connected node
    connected_nodes_info = []
    for record in results:
        name = record["name"]
        desc = record["desc"]
        connected_nodes_info.append({"Name": name, "Desc": desc})
    
    return connected_nodes_info

In [ ]:
from langchain.prompts import PromptTemplate

choosing_template = """
You are an agent with multiple other AI agents for your assistance.
Given a list of connected agents with their names and description, if required return the name of the next agent you require for your assistance as well as give an instruction for it.
List of agents : {Connected_agents} 
{agent_scratchpad}
AI: 
"""

# Initialize the prompt template
choosing_prompt = PromptTemplate(input_variables=["Connected_agents"], template=choosing_template)